## Install necessary libraries

In [1]:
!pip install matplotlib
!pip install pandas
!pip install mercury
!pip install --upgrade mergetb

## Import necessary libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time, socket
from timeout_timer import timeout, TimeoutInterrupt

from mergetb.workspace import User, Experiment, Project
from mergetb.realize import Realization
from mergetb.materialize import Materialization
from mergetb.grpc_client import MergeGRPCError
from mergetb.types import StatusType

## Log in as user

In [3]:
username='alix'
passw='SmaduMestivel13'

try:
    u = User(username, passw)
    u.login()
except MergeGRPCError as e:
    print(f"Error logging in: {e}")
else:
    print("Logged in as user", u.username)

  [stream_terminated: attempting try 2 out of 5]
Logged in as user alix


## Find or create experiment

In [4]:
try:
    # load the user's personal project
    p = Project(username)
    resp = p.get()

    # look for the experiment, create if it does not exist
    for exp in resp.project.experiments:
        if exp == 'dpra':
            print('experiment dpra already exists')
            break
    else:
        e = Experiment('dpra', username, description='Replicating DPRA artifact')
        e.create()
        print('experiment dpra created')
    
except MergeGRPCError as e:
    print(e)
#proj = m.find_project(username)
#exp = m.find_experiment('dpra', proj.name)
#if exp == None:
#    exp = m.create_experiment('dpra', proj.name, 'Replicating DPRA artifact')

experiment dpra created


## Push model, allocate resources and attach the experiment to the XDC

In [5]:
RLZ_TIMEOUT=15
MTZ_TIMEOUT=60

try:
    exp = Experiment('dpra', username)
    rev = exp.push_model('1726854460567.py').revision
    print("Pushed experiment revision", rev)
    time.sleep(5)
    
    # reserve resources through a lease named 'v1' (name is arbitrary)
    exp.reserve('v2', revision=rev)

    # wait up for 10 sec for reservation to succeed
    try:
        rlz = Realization('v2', exp.name, exp.project)
        with timeout(RLZ_TIMEOUT):
            while True:
                resp = rlz.get()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('leased resources')
                    break
    except TimeoutInterrupt:
        raise Exception('reservation did not succeed within %d seconds' % RLZ_TIMEOUT)

    # activate experiment on this lease
    exp.activate('v2')

    # wait up to 60 sec for activation to succeed
    try:
        mtz = Materialization('v2', exp.name, exp.project)
        with timeout(MTZ_TIMEOUT):
            while True:
                resp = mtz.get_status()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('started experiment')
                    break
    except TimeoutInterrupt:
        raise Exception('activation did not succeed within %d seconds' % MTZ_TIMEOUT)

    hostname = socket.gethostname().split('-')[0].split('.')[0]
    exp.attach_xdc('v2', hostname, username)
    time.sleep(15)
    print("Attached experiment is now running")
except MergeGRPCError as e:
    print(e)

Pushed experiment revision b659debcbe69d40a1a92283922c5dcbcb2e9054a
leased resources
  [stream_terminated: attempting try 2 out of 5]
started experiment
Attached experiment is now running


## Run commands in experiment to clone the artifact, install prerequisites, fix some bugs and execute one script in the artifact

In [6]:
installpackages="sudo apt update && sudo apt install git python-is-python3 python3-pip -y"
clonerepo="git clone https://github.com/dpra-dp/dpra.git"
installpip="pip install numpy matplotlib"
fixartifact="cd dpra; sed -i 's/geo1.pdf/cst.pdf/g' cst.py; sed -i 's/seaborn/ggplot/g' *.py; sed -i '62s/# *//' geo2.py; sed -i 's/s=s/s=0/g' uni.py; sed '108i plt.savefig(\"utility.pdf\", bbox_inches=\"tight\")' theo_utility_comparison.py > theo_utility_comparison_new.py"
print("Installing Linux packages, this step may take a while ...")
(lines, elines) = exp.exec_on_node(username, "nodex", installpackages)
for line in lines:
    print(line.strip())
print("Cloning artifact repository");
(lines, elines) = exp.exec_on_node(username, "nodex", clonerepo)
for line in lines:
     print(line.strip())
print("Installing Python packages")
(lines, elines) = exp.exec_on_node(username, "nodex", installpip)
for line in lines:
     print(line.strip())
    
print("Applying artifact fixes")
(lines, elines) = exp.exec_on_node(username, "nodex", fixartifact)
for line in lines:
     print(line.strip())
print("Will now execute the code for theoretical utility comparison")
runcode1="cd dpra; python theo_utility_comparison_new.py"
(lines, elines) = exp.exec_on_node(username, "nodex", runcode1)
for line in lines:
     print(line.strip())



Installing Linux packages, this step may take a while ...
Get:1 http://security.debian.org/debian-security bullseye-security InRelease [27.2 kB]
Get:2 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://security.debian.org/debian-security bullseye-security/main Sources [187 kB]
Get:5 http://security.debian.org/debian-security bullseye-security/main amd64 Packages [296 kB]
Get:6 http://security.debian.org/debian-security bullseye-security/main Translation-en [190 kB]
Get:7 http://deb.debian.org/debian bullseye/contrib Sources [43.2 kB]
Get:8 http://deb.debian.org/debian bullseye/main Sources [8,500 kB]
Get:9 http://deb.debian.org/debian bullseye/non-free Sources [81.0 kB]
Get:10 http://deb.debian.org/debian bullseye/main amd64 Packages [8,066 kB]
Get:11 http://deb.debian.org/debian bullseye/main Translation-en [6,235 kB]
Get:12 http://deb.debian.org/debian bullseye/non-free amd64 Packages [96.4 kB]

In [7]:
print("Will now execute the code for geo1")
runcode2="cd dpra; python geo1.py"
(lines, elines) = exp.exec_on_node(username, "nodex", runcode2)
for line in lines:
     print(line.strip())
print("end")

Will now execute the code for geo1
0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.000000000000001
8.0
9.0
10.0
11.0
12.0
13.0
14.000000000000002
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.000000000000004
28.999999999999996
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.00000000000001
56.00000000000001
56.99999999999999
57.99999999999999
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
0.9
max index before RA (y): 5
geometric - Simulation (k = 10), before RA: 2.3519101595565473
max index (y): 0
geometric - Simulation (k = 10), after RA: 3.5553480614894135
geometric - Simulation (k = 10), before RA (keep inf): inf
geometric - Simulation (k = 10), after RA (keep inf): inf
utility: 0.990081662999995
[0, 0, 0, 0, 1, 114, 2217, 42262, 658532, 8465043,

In [8]:
print("Will now execute the code for akr")
runcode2="cd dpra; python akr.py"
(lines, elines) = exp.exec_on_node(username, "nodex", runcode2)
for line in lines:
     print(line.strip())
print("end")

Will now execute the code for akr
0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7000000000000001
0.8
0.8999999999999999
1.0
1.0999999999999999
1.2
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.7999999999999998
1.9
2.0
2.1
2.1999999999999997
2.3
2.4
2.5
2.6
2.7
2.8000000000000003
2.9000000000000004
3.0
3.1
3.2
3.3000000000000003
3.4000000000000004
3.5000000000000004
3.5999999999999996
3.6999999999999997
3.8
3.9
4.0
4.1000000000000005
4.2
4.3
4.3999999999999995
4.5
4.6
4.7
4.8
4.9
5.0
5.1
5.2
5.3
5.4
5.5
5.6000000000000005
5.7
5.800000000000001
5.8999999999999995
6.0
6.1
6.2
6.3
6.4
6.5
6.6000000000000005
6.7
6.800000000000001
6.9
7.000000000000001
7.1
7.199999999999999
7.3
7.3999999999999995
7.5
7.6
7.7
7.8
7.9
8.0
8.1
8.200000000000001
8.3
8.4
8.5
8.6
8.7
8.799999999999999
8.9
9.0
9.1
9.2
9.3
9.4
9.5
9.6
9.700000000000001
9.8
9.9
10.0
10.100000000000001
10.2
10.299999999999999
10.4
10.5
10.6
10.7
10.8
10.9
11.0
11.1
11.200000000000001
11.3
11.4
11.5
11.600000000000001
11.700000000000001
11.79999

In [10]:
print("Will now execute the code for uni")
runcode2="cd dpra; python uni.py"
(lines, elines) = exp.exec_on_node(username, "nodex", runcode2)
for line in lines:
     print(line.strip())
print("end")

Will now execute the code for uni
-10 10
x_l, x_r -10 10
uniform - Simulation (k = 10), before RA: 0.0017134820261790426
max (y) at: 0
uniform - Simulation (k = 10), after RA: 2.3970666145655293
utility: 0.5803655880000013
-10 15
x_l, x_r -10 15
uniform - Simulation (k = 10), before RA: 0.0011917806796570855
max (y) at: 0
uniform - Simulation (k = 10), after RA: 2.3631009710743305
utility: 0.5526946879999994
end


In [11]:
print("Will now execute the code for cst")
runcode2="cd dpra; python cst.py"
(lines, elines) = exp.exec_on_node(username, "nodex", runcode2)
for line in lines:
     print(line.strip())
print("end")

Will now execute the code for cst
0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.000000000000001
8.0
9.0
10.0
11.0
12.0
13.0
14.000000000000002
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.000000000000004
28.999999999999996
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.00000000000001
56.00000000000001
56.99999999999999
57.99999999999999
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
0.9
max index (y): 0
constant - Simulation (k = 10), after RA: 1.6530332597264283
constant - Simulation (k = 10), before RA (keep inf): inf
constant - Simulation (k = 10), after RA (keep inf): 1.6530332597264283
utility: 0.4999944569999984
[579, 54144, 1096479, 7793883, 23873805, 34366186, 23871199, 7792637, 1096424, 54129, 535] [3024, 155629, 2106003, 11224296, 275

In [12]:
print("Will now execute the code for geo2")
runcode="cd dpra; python geo2.py"
(lines, elines) = exp.exec_on_node(username, "nodex", runcode)
for line in lines:
     print(line.strip())
print("end")

Will now execute the code for geo2
0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.000000000000001
8.0
9.0
10.0
11.0
12.0
13.0
14.000000000000002
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.000000000000004
28.999999999999996
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.00000000000001
56.00000000000001
56.99999999999999
57.99999999999999
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
(miu,s) =  0 1
max index before RA (y): 23
geometric2 - Simulation (k = 10), before RA: 1.1019071845649622
max index (y): 10
geometric2 - Simulation (k = 10), after RA: 2.0725525645751164
utility: 0.9220987479999985
end


## Copy graph as PDF onto XDC

In [13]:
import subprocess
process1 = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodex:dpra/utility.pdf /tmp/utility.pdf &> /dev/null", shell=True, stdout=subprocess.DEVNULL)
process2 = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodex:dpra/geo1.pdf /tmp/geo1.pdf &> /dev/null", shell=True, stdout=subprocess.DEVNULL)
process3 = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodex:dpra/uni.pdf /tmp/uni.pdf &> /dev/null", shell=True, stdout=subprocess.DEVNULL)
process4 = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodex:dpra/akr.pdf /tmp/akr.pdf &> /dev/null", shell=True, stdout=subprocess.DEVNULL)
process5 = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodex:dpra/geo2.pdf /tmp/geo2.pdf &> /dev/null", shell=True, stdout=subprocess.DEVNULL)
process6 = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodex:dpra/cst.pdf /tmp/cst.pdf &> /dev/null", shell=True, stdout=subprocess.DEVNULL)


## Display graph

In [14]:
import mercury as mr
mr.PDF("/tmp/utility.pdf")

In [15]:
mr.PDF("/tmp/geo1.pdf")

In [16]:
mr.PDF("/tmp/akr.pdf")

In [17]:
mr.PDF("/tmp/uni.pdf")

In [19]:
mr.PDF("/tmp/cst.pdf")

In [20]:
mr.PDF("/tmp/geo2.pdf")

## Return resources and if you so wish delete the experiment

In [ ]:
#try:
#    exp.relinquish('v1')
#except MergeGRPCError as e:
#    print(e)
#else:
#    print("experiment relinquished")
#exp.delete()